# Model Assessment

In [9]:
from tqdm.notebook import tqdm
import pickle, gzip
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from scipy.special import softmax
import torch

import nibabel as nib
from nilearn import maskers
from nilearn.plotting import view_img
from nilearn.image import resample_img

from neurovlm.data import get_data_dir
from neurovlm.models import Specter

from neurovlm.brain_input import search_papers_from_brain, search_wiki_from_brain

In [10]:
get_data_dir()

with gzip.open(get_data_dir() / f"networks_arrays.pkl.gz", "rb") as f:
    networks = pickle.load(f)

proj_head_mse_adhoc = torch.load(get_data_dir() / f"proj_head_image_infonce.pt", weights_only=False).cpu()
proj_head_img = torch.load(get_data_dir() / f"proj_head_image_infonce.pt", weights_only=False).cpu()
proj_head_text = torch.load(get_data_dir() / f"proj_head_text_infonce.pt", weights_only=False).cpu()
specter = Specter("allenai/specter2_aug2023refresh", adapter="adhoc_query")
autoencoder = torch.load(get_data_dir() / "autoencoder_sparse.pt", weights_only=False).cpu()
# decoder = autoencoder.decoder.to("cpu")
mask_arrays = np.load(f"{get_data_dir()}/mask.npz", allow_pickle=True)
mask_img = nib.Nifti1Image(mask_arrays["mask"].astype(float),  mask_arrays["affine"])
masker = maskers.NiftiMasker(mask_img=mask_img, dtype=np.float32).fit()

There are adapters available but none are activated for the forward pass.


## Network vs. articles

In [3]:
networks = {k: v for _k in networks.keys() for k, v in networks[_k].items()}

In [4]:
mask_arrays = np.load(f"{get_data_dir()}/mask.npz", allow_pickle=True)
mask_img = nib.Nifti1Image(mask_arrays["mask"].astype(float),  mask_arrays["affine"])
masker = maskers.NiftiMasker(mask_img=mask_img, dtype=np.float32).fit()
networks_resampled = {}

for k in tqdm(networks.keys(), total=len(networks)):
    img = nib.Nifti1Image(networks[k]["array"], affine=networks[k]["affine"])

    if len(np.unique(networks[k]["array"])) == 2:
        # binary data
        img_resampled = resample_img(img, mask_arrays["affine"], interpolation="nearest")
    else:
        img_resampled = resample_img(img, mask_arrays["affine"])
        img_resampled_arr = img_resampled.get_fdata()
        img_resampled_arr[img_resampled_arr < 0] = 0.
        thresh = np.percentile(img_resampled_arr.flatten(), 95)
        img_resampled_arr[img_resampled_arr < thresh] = 0.
        img_resampled_arr[img_resampled_arr >= thresh] = 1.
        img_resampled = nib.Nifti1Image(img_resampled_arr, affine=mask_arrays["affine"])

    networks_resampled[k] = img_resampled

  0%|          | 0/152 [00:00<?, ?it/s]

In [5]:
networks_embed = {}

for k, v in tqdm(networks_resampled.items(), total=len(networks_resampled)):
    networks_embed[k] = autoencoder.encoder(torch.from_numpy(masker.transform(v)))

  0%|          | 0/152 [00:00<?, ?it/s]

In [7]:
networks_embed.keys()

dict_keys(['VIS-P', 'CG-OP', 'DN-B', 'SMOT-B', 'AUD', 'PM-PPr', 'dATN-B', 'SMOT-A', 'LANG', 'FPN-B', 'FPN-A', 'dATN-A', 'VIS-C', 'SAL/PMN', 'DN-A', 'NONE', 'Visual1', 'Visual2', 'Somatomotor', 'CingOperc', 'DorsAttn', 'Language', 'FrontPar', 'Auditory', 'Default', 'PostMulti', 'VentMulti', 'OrbitAffective', 'ICA1', 'ICA2', 'ICA3', 'ICA4', 'ICA5', 'ICA6', 'ICA7', 'ICA8', 'ICA9', 'ICA10', 'ICA11', 'ICA12', 'ICA13', 'ICA14', 'ICA15', 'ICA17', 'ICA18', 'ICA19', 'ICA20', 'ICA23', 'Emo/Interoception1', 'Emo/Interoception2', 'Emo/Interoception3', 'Emo/Interoception4', 'Mot/Visspatial1', 'Mot/Visspatial2', 'Mot/Visspatial3', 'Mot/Visspatial4', 'Visual3', 'DivergentCog1', 'DivergentCog3', 'DivergentCog4', 'DivergentCog5', 'DivergentCog6', 'medial frontal', 'frontoparietal', 'default mode', 'motor cortex', 'visual A', 'visual B', 'visual association', 'subcortical cerebellum', 'AntSal', 'DorsalDMN', 'HighVisual', 'LECN', 'PostSal', 'Precuneus', 'PrimVisual', 'RECN', 'Sensorimotor', 'VentralDMN',